In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from datetime import datetime

In [2]:
# Load the data
from google.colab import files
uploaded = files.upload()

Saving IPL_ball_by_ball_updated till 2024 (1).csv to IPL_ball_by_ball_updated till 2024 (1).csv


In [23]:
ipl_data = pd.read_csv('/content/IPL_ball_by_ball_updated till 2024 (1).csv')

<ipython-input-23-e2ccbc442003>:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  ipl_data = pd.read_csv('/content/IPL_ball_by_ball_updated till 2024 (1).csv')


In [25]:
# Filter data for R. Ashwin's performance
ashwin_data = ipl_data[ipl_data['Bowler'] == 'R Ashwin']

# Convert 'Date' to datetime format and extract year
ashwin_data['Date'] = pd.to_datetime(ashwin_data['Date'], format='%d-%m-%Y')
ashwin_data['Year'] = ashwin_data['Date'].dt.year

# Filter data for the last three seasons (2022, 2023, 2024)
last_three_years = ashwin_data[ashwin_data['Season'].isin(['2022', '2023', '2024'])]

# Function to calculate performance metrics
def calculate_metrics(data):
    wickets = len(data[data['wicket_confirmation'] == 1])
    runs_conceded = data['runs_scored'].sum()
    balls_bowled = len(data)
    overs_bowled = balls_bowled / 6
    economy_rate = runs_conceded / overs_bowled if overs_bowled != 0 else 0
    return pd.Series({'Wickets': wickets, 'RunsConceded': runs_conceded, 'OversBowled': overs_bowled, 'EconomyRate': economy_rate})

# Calculate performance metrics for each season
metrics = last_three_years.groupby('Season').apply(calculate_metrics).reset_index(drop=True)

# Add the Season column back
metrics['Season'] = last_three_years['Season'].unique()

# Add salary data for R. Ashwin (example values, replace with actual salary data)
salary_data = pd.DataFrame({
    'Season': ['2022', '2023', '2024'],
    'Salary': [76000000, 77000000, 78000000]  # in INR
})

# Merge performance metrics with salary data
analysis_data = pd.merge(metrics, salary_data, on='Season')

<ipython-input-25-268f9e8b6fa7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ashwin_data['Date'] = pd.to_datetime(ashwin_data['Date'], format='%d-%m-%Y')
<ipython-input-25-268f9e8b6fa7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ashwin_data['Year'] = ashwin_data['Date'].dt.year


In [27]:
# Print analysis_data columns to debug
print("Analysis DataFrame columns:", analysis_data.columns)

Analysis DataFrame columns: Index(['Match id', 'Date', 'Batting team', 'Bowling team', 'Innings No',
       'Ball No', 'Bowler', 'Striker', 'Non Striker', 'runs_scored', 'extras',
       'type of extras', 'score', 'score/wicket', 'wicket_confirmation',
       'wicket_type', 'fielders_involved', 'Player Out', 'Year', 'Season',
       'Salary'],
      dtype='object')


In [29]:
# Perform regression analysis
X = analysis_data['Season'].astype(int)  # Convert 'Season' to integer type
y = analysis_data['Salary']

# Adding a constant for statsmodels
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()
print(model.summary())

# Plot the relationships
# Wickets vs Salary
plt.figure(figsize=(10, 6))
sns.regplot(x='Wickets', y='Salary', data=analysis_data)
plt.title('Relationship between Wickets and Salary')
plt.xlabel('Wickets')
plt.ylabel('Salary (INR)')
plt.show()

# Economy Rate vs Salary
plt.figure(figsize=(10, 6))
sns.regplot(x='EconomyRate', y='Salary', data=analysis_data)
plt.title('Relationship between Economy Rate and Salary')
plt.xlabel('Economy Rate')
plt.ylabel('Salary (INR)')
plt.show()

ValueError: zero-size array to reduction operation maximum which has no identity